In [1]:
import pathlib

import pandas as pd

In [2]:
file_path = pathlib.Path(
    "../../data/5.converted_data/custom_aggregated_data.parquet"
).resolve(strict=True)
df = pd.read_parquet(file_path)
df.head()

,Metadata_genotype,Metadata_replicate,AreaShape_Area,AreaShape_CentralMoment_0_0,AreaShape_CentralMoment_0_1,AreaShape_CentralMoment_0_2,AreaShape_CentralMoment_0_3,AreaShape_CentralMoment_1_0,AreaShape_CentralMoment_1_1,AreaShape_CentralMoment_1_2,...,Texture_SumEntropy_OP_3_02_256,Texture_SumEntropy_OP_3_03_256,Texture_SumVariance_OP_3_00_256,Texture_SumVariance_OP_3_01_256,Texture_SumVariance_OP_3_02_256,Texture_SumVariance_OP_3_03_256,Texture_Variance_OP_3_00_256,Texture_Variance_OP_3_01_256,Texture_Variance_OP_3_02_256,Texture_Variance_OP_3_03_256
0,high,1,1.192966,0.596483,0.541753,-0.049237,0.781736,2.100915,-0.927874,-0.042982,...,-0.208210,-0.119036,-1.091712,-0.525027,-1.086609,-0.544589,-1.115127,-0.562101,-1.117953,-0.560872
1,high,10,2.477908,1.238954,1.319626,1.722364,-1.748595,0.179195,-3.289519,0.825270,...,1.293954,0.635356,1.261364,0.642091,1.187508,0.635274,1.115726,0.548473,1.121969,0.547769
2,high,11,4.262414,2.131207,1.261731,1.426469,0.583182,1.801207,-0.541325,-0.164307,...,1.373542,0.678230,1.104765,0.570823,1.052532,0.548475,0.997651,0.486269,1.000166,0.490213
3,high,12,2.402476,1.201238,0.049886,1.279119,-0.928171,-1.652853,-1.964619,0.107680,...,0.619009,0.308933,-0.109557,-0.059876,-0.196800,-0.067930,-0.207564,-0.109367,-0.202171,-0.106772
4,high,13,2.389777,1.194888,-1.108736,-0.034466,0.101765,-1.344980,-0.967345,-0.794434,...,0.113413,0.047741,-0.650341,-0.295674,-0.642077,-0.322436,-0.696817,-0.356255,-0.700572,-0.352168


We want to remove CellProfiler features that do not accurately contribute to the shape variance.


Spatial moment features are removed as they are not invariant to rotation and translation.

Central moment features are removed as measurements of size is not of interest.

Normalized moment features are removed as they do not contribute to the shape variance.

EulerNumber is removed as we are not concerned with the number of holes in the shape.

Extent is removed as we do not care about the ratio of pixels in the bounding box to the pixels in the shape.

Orientation is removed as we are not concerned with the angle of the major axis of the shape.

Zernike features are removed as they are not invariant to rotation and translation.

Texture features are removed as we are not interested in the texture of the shape.

In [3]:
list_to_drop = [
    "SpatialMoment",
    "CentralMoment",
    "NormalizedMoment",
    "EulerNumber",
    "Extent",
    "Orientation",
    "Zernike",
    "Texture",
]

In [4]:
# drop columns that contain Metadata
df1 = df.copy()
df1 = df1.drop(columns=[col for col in df1.columns if "Metadata" in col])
columns = df1.columns.to_list()
print(df1.shape)
# if column contains any of the strings in list_to_drop, drop it
for col in list_to_drop:
    columns = [c for c in columns if col not in c]
df1 = df1[columns]
print(df1.shape)
df1["Metadata_genotype"] = df["Metadata_genotype"]
df1["Metadata_replicate"] = df["Metadata_replicate"]

(42, 244)
(42, 64)


In [5]:
df1.head()

,AreaShape_Area,AreaShape_Compactness,AreaShape_ConvexArea,AreaShape_Eccentricity,AreaShape_EquivalentDiameter,AreaShape_FormFactor,AreaShape_HuMoment_0,AreaShape_HuMoment_1,AreaShape_HuMoment_2,AreaShape_HuMoment_3,...,RadialDistribution_MeanFrac_OP_1of4,RadialDistribution_MeanFrac_OP_2of4,RadialDistribution_MeanFrac_OP_3of4,RadialDistribution_MeanFrac_OP_4of4,RadialDistribution_RadialCV_OP_1of4,RadialDistribution_RadialCV_OP_2of4,RadialDistribution_RadialCV_OP_3of4,RadialDistribution_RadialCV_OP_4of4,Metadata_genotype,Metadata_replicate
0,1.192966,0.461635,1.744956,-2.085548,1.348456,-0.003732,-1.438887,-0.669940,-1.604103,-0.642703,...,0.208840,-0.038983,-0.205642,0.180570,-1.659848,-0.748209,-0.734737,-0.447147,high,1
1,2.477908,-1.084807,2.280730,-0.097805,2.496823,1.278626,-1.530468,-0.608788,-1.845856,-0.641467,...,-2.497710,-1.375573,-2.599793,1.556094,1.782285,0.624669,2.211775,1.402458,high,10
2,4.262414,-0.755467,4.433965,-1.886368,3.782522,0.795809,-1.848755,-0.685352,-1.922923,-0.660985,...,0.143854,-0.521327,-1.791332,0.728454,1.205187,0.579514,1.059906,0.684432,high,11
3,2.402476,-1.171680,2.190627,-0.296606,2.427407,1.488535,-1.480895,-0.592887,-1.731990,-0.607569,...,-2.176503,-1.243669,-1.981686,1.272853,-0.908438,-0.089235,1.397076,0.836480,high,12
4,2.389777,-0.952083,2.200426,0.052727,2.394630,1.025243,-1.305085,-0.548787,-1.570746,-0.525058,...,-2.530155,-1.415683,-2.696436,1.679224,-1.159343,-0.938057,0.004541,0.626942,high,13


In [6]:
# save the dataframe
df_path = pathlib.Path(
    "../../data/5.converted_data/custom_aggregated_manual_feature_selected_data.parquet"
)
df1.to_parquet(df_path)